In [1]:
import re
import cv2
from pyzbar.pyzbar import *

In [2]:
# 입력소스 선택

# cap = cv2.VideoCapture(0)                            # 첫번째 카메라
cap = cv2.VideoCapture("QRvideo1.MOV")                 # 동영상 파일
# cap = cv2.VideoCapture("rtsp://ID:PW@ip-address")    # IP카메라

In [3]:
cap.isOpened()    # 카메라가 없어도 True일 수 있음 ==> read()에서 False

True

In [4]:
# 입력소스의 속성
# cap.get(cv2.CAP_PROP_FRAME_WIDTH) = 가로 해상도(3)
# cap.get(cv2.CAP_PROP_FRAME_HEIGHT) = 세로 해상도(4)
# cap.get(cv2.CAP_PROP_FRAME_COUNT) = 총 프레임 수
# cap.get(cv2.CAP_PROP_FPS) = 프레임율(5)

print('Frame width:', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height:', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('Frame count:', int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print('FPS:', cap.get(cv2.CAP_PROP_FPS))

Frame width: 3840
Frame height: 2160
Frame count: 343
FPS: 59.96503496503497


In [5]:
# 입력소스 = 카메라인 경우: 해상도, 프레임률 변경

# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
# cap.set(cv2.CAP_PROP_FPS, 30)

In [31]:
# <QR Format>
# 식당: NWTAVR-MA-202111-식당이름
# 직원: 2021-11-4077-01 ==> 다음달부터 NWTAVR-MS-202112-4077-01

# HEADER = 노원구청 교통행정과 등록팀 (내 QR인지 식별자 역할)
# CLASS : 가맹점 = Member of Affiliation / 직원 = Member of Staff
QR_HEADER = "NWTAVR"        # 6
QR_CLASS_MA = "MA"          # 2
QR_CLASS_MS = "MS"          # 2
QR_YEAR = "2021"                      # 4
QR_MONTH = "11"                       # 2
QR_YEAR_MONTH = QR_YEAR + QR_MONTH    # 6
QR_SN = ["0"+str(i) for i in range(1,10)] + list(map(str, range(10, 21)))    # ["01", "02", ... "19", "20"]

affil_list = ["노란코끼리", "어장촌생선구이", "횡성목장", "항도(港都)", "전주콩나루", "옛날칼국수", "칠리사이공", "도나한우", "북경(北京)", "명문식당", "일성스시", "새싹비빔밥", "구내매점"]

staff_list = ["전현호", "박선녕", "정성욱", "이재영", "김재형", "이희영", "신상용", "최미연", "이정선"]
phone_list = ["4080",   "4079",   "4076",   "4075",   "4077",   "4071",   "4069",   "4068",   "4078"]
color_list = ["#E32636", "#FF5511", "#FFA500", "#FCE205", "#119617", "#AAEE00", "#0FF1C3", "#1155FF", "#AB3ED8"]


MA_pattern = re.compile(QR_HEADER + "-" + QR_CLASS_MA + "-" + QR_YEAR_MONTH + "-" + ".+")
MS_pattern = re.compile(QR_YEAR_MONTH[:4] + "-" + QR_YEAR_MONTH[4:] + "-" + "[0-9]{4}-[0-9]{2}")

In [31]:
# 필요한 QR만 선택
# 직원 QR만 전처리 ==> 가맹점QR은 1개뿐이므로 전처리 불필요


preprocessed_QR_list = []

def PreprocessFrame(frame):
	
	# 프레임에서 모든 QR code 찾기
	QR_list = decode(frame, symbols=[ZBarSymbol.QRCODE])
	num_of_QR = len(QR_list)
	print(f"[전처리 전] QR개수 : {num_of_QR}")
	
	# 필요한 QR만 골라서 decoded_list에 넣음
	for each_QR in QR_list:
		# QR 형식검증1 : data 길이=15
		if len(each_QR.data) != 15:
			result.remove(each_QR)
			continue

		# QR 형식검증2 : 패턴("2021-11-4077-01")
		QRdata_str = each_QR.data.decode('utf-8')
		match_result = MS_pattern.match(QRdata_str)
		if match_result is None:
			result.remove(each_QR)
			continue
		else:
			QR_contents = match_result.group()

		# QR 내용검증 : 연/월/전화번호/일련번호
		yyyy, mm, staff, sn = QR_content.split('-')
		if (yyyy != QR_YEAR) or (mm != QR_MONTH) or (staff not in phone_list) or (sn not in QR_SN):
			result.remove(each_QR)
			continue

		# 모두 통과했다면 필요한 QR
		preprocessed_QR_list.append(each_QR)
	
	QR_count = len(preprocessed_QR_list)
	print(f"[전처리 후] QR개수 : {QR_count}")
	return QR_count

True

In [ ]:
detected_QR_list = []
frame_id = 0

while True:
	# 열려있는 카메라가 없으면 이 시점에서 ret = False
	read_success, frame = cap.read()
	if read_success == False:
		break
	
	# 카메라 창 띄움
	# frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	# cv2.imshow("Frame", frame_RGB)
	cv2.imshow("Frame", frame)
	
	# ESC 키가 눌려지면 루프 탈출
	key = cv2.waitKey(100) & 0xFF
	if key == 27:
		cap.release()
		cv2.destroyAllWindows()
		break
		
	# 프레임에서 모든 QR code 찾기
	result = decode(frame, symbols=[ZBarSymbol.QRCODE])
	num_of_QR = len(result)
	print(f"[전처리 전] QR개수 : {num_of_QR}")
	

	
	
	